In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 98% !important; }</style>"))
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [ ]:
import sys
import os
import re
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm
from sklearn.metrics import f1_score

In [ ]:
from lib import util, embeds, fitter, masker, features, sentiment, translate

In [ ]:
datapath = 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data'
testpath = 'SemEval_2022_Task2-idiomaticity/SubTaskA/TestData'
multilingual_model = 'distiluse-base-multilingual-cased-v1'

In [ ]:
frames = util.load_csv_dataframes(datapath)
tframes = util.load_csv_dataframes(testpath)

In [ ]:
zdf = frames['train_zero_shot.csv']
odf = frames['train_one_shot.csv']
ddf = frames['dev.csv']
ddf_gold = frames['dev_gold.csv']
edf = frames['eval.csv']
tdf = tframes['test.csv']

In [ ]:
z_emb_multi = embeds.get_embeddings(zdf, modelname=multilingual_model, append=['MWE'])
d_emb_multi = embeds.get_embeddings(ddf, modelname=multilingual_model, append=['MWE'])
e_emb_multi = embeds.get_embeddings(edf, modelname=multilingual_model, append=['MWE'])
t_emb_multi = embeds.get_embeddings(tdf, modelname=multilingual_model, append=['MWE'])

In [ ]:
z_score, z_probs, z_results = fitter.get_fit_results(z_emb_multi, zdf['Label'], d_emb_multi, ddf_gold['Label'])

In [ ]:
z_score

In [ ]:
f1_score(z_results, ddf_gold['Label'], average='macro')

Using only sentence-transformers embeddings results in 61.4% score.

Create a dataframe for the results.

In [ ]:
resdf_dev = pd.DataFrame(columns=['Name', 'Score'])

In [ ]:
resdf_dev

In [ ]:
resdf_dev.loc['sbert'] = ['Sentence transformers', f1_score(z_results, ddf_gold['Label'], average='macro')]

### Reload stored data

In [ ]:
zdf_bt3 = pd.read_pickle('data/zdf_bt3_20220104_1.pkl')
ddf_bt3 = pd.read_pickle('data/ddf_bt3_20220104_1.pkl')
edf_bt3 = pd.read_pickle('data/edf_bt3_20220105_1.pkl')
tdf_bt3 = pd.read_pickle('data/tdf_bt3_20220111_1.pkl')

In [ ]:
edf_sub = frames['eval_submission_format.csv']
tdf_sub = tframes['test_submission_format.csv']

In [ ]:
# dropcols = ['Top score', 'FS', 'SS', 'Quotes', 'MWEdiff']
# dropcols = ['Hassub', 'FS', 'Nextdiff']
# dropcols = ['MWEdiff', 'FS']
# dropcols = ['Top score 1', 'Top score 2', 'SS', 'FS', 'MWEdiff']
dropcols = ['Top score 1', 'FS', 'MWEdiff']
zdf_t4 = fitter.get_trainable(zdf_bt3).drop(dropcols, axis=1)
ddf_t4 = fitter.get_trainable(ddf_bt3).drop(dropcols, axis=1)
ddf5_feat_score, ddf5_feat_probs, ddf5_feat_results = fitter.get_fit_results(zdf_t4, zdf['Label'], ddf_t4, ddf_gold['Label'])

In [ ]:
mup = fitter.multi_results(ddf_bt3, ddf_gold, z_results, z_probs, ddf5_feat_results, ddf5_feat_probs, ['Caps', 'Hassub'],['Quotes'])
f1_score(mup['Prediction'], mup['Label'], average='macro')

In [ ]:
resdf_dev.loc['sbert_feat'] = ['Sentence transformers + feature', f1_score(mup['Prediction'], mup['Label'], average='macro')]

In [ ]:
ddf_sub = frames['dev_submission_format.csv']
ddf_res = ddf_sub.copy()
ddf_res.loc[ddf_res['Setting'] == 'zero_shot', 'Label'] = mup['Prediction']
ddf_res.to_csv('data/ddf_sub_20220121_1.csv', index=False)

#### Normalization of sentiment scores

Trying to see if normalizing sentiment helps.

In [ ]:
print(zdf_bt3[zdf_bt3['Language'] == 'EN']['Sentiment'].mean())
print(zdf_bt3[zdf_bt3['Language'] == 'PT']['Sentiment'].mean())
sentdiff = zdf_bt3[zdf_bt3['Language'] == 'PT']['Sentiment'].mean() - zdf_bt3[zdf_bt3['Language'] == 'EN']['Sentiment'].mean()
print(sentdiff)

In [ ]:
zdf_bt4 = zdf_bt3.copy()
zdf_bt4['SentNorm'] = zdf_bt4['Sentiment']
zdf_bt4.loc[zdf_bt4['Language'] == 'PT', 'SentNorm'] -= sentdiff

In [ ]:
print(ddf_bt3[ddf_bt3['Language'] == 'EN']['Sentiment'].mean())
print(ddf_bt3[ddf_bt3['Language'] == 'PT']['Sentiment'].mean())
sentdiff_d = ddf_bt3[ddf_bt3['Language'] == 'PT']['Sentiment'].mean() - ddf_bt3[ddf_bt3['Language'] == 'EN']['Sentiment'].mean()
print(sentdiff_d)

In [ ]:
ddf_bt4 = ddf_bt3.copy()
ddf_bt4['SentNorm'] = ddf_bt4['Sentiment']
ddf_bt4.loc[ddf_bt4['Language'] == 'PT', 'SentNorm'] -= sentdiff

In [ ]:
dropcols_s = ['Top score 1', 'FS', 'MWEdiff', 'Sentiment']
zdf_t6 = fitter.get_trainable(zdf_bt4).drop(dropcols_s, axis=1)
ddf_t6 = fitter.get_trainable(ddf_bt4).drop(dropcols_s, axis=1)
ddf7_feat_score, ddf7_feat_probs, ddf7_feat_results = fitter.get_fit_results(zdf_t6, zdf['Label'], ddf_t6, ddf_gold['Label'])
mup_s = fitter.multi_results(ddf_bt3, ddf_gold, z_results, z_probs, ddf7_feat_results, ddf7_feat_probs, ['Caps', 'Hassub'],['Quotes'])
f1_score(mup_s['Prediction'], mup_s['Label'], average='macro')

Normalization of the Sentiment scores doesn't seem to help.

In [ ]:
zdf_bt5 = pd.get_dummies(zdf_bt3, columns=['Language'])
#zdf_bt5['OneHotEN'] = zdf_bt5['OneHotPT'] = zdf_bt5['OneHotGL'] = 0
#zdf_bt5.loc[zdf_bt5['Language'] == 'EN', 'OneHotEN'] = 1
#zdf_bt5.loc[zdf_bt5['Language'] == 'PT', 'OneHotPT'] = 1
#zdf_bt5.loc[zdf_bt5['Language'] == 'GL', 'OneHotGL'] = 1
ddf_bt5 = pd.get_dummies(ddf_bt3, columns=['Language'])
ddf_bt5 = ddf_bt3.copy()
#ddf_bt5['OneHotEN'] = ddf_bt5['OneHotPT'] = ddf_bt5['OneHotGL'] = 0
#ddf_bt5.loc[ddf_bt5['Language'] == 'EN', 'OneHotEN'] = 1
#ddf_bt5.loc[ddf_bt5['Language'] == 'PT', 'OneHotPT'] = 1
#ddf_bt5.loc[ddf_bt5['Language'] == 'GL', 'OneHotGL'] = 1

In [ ]:
dropcols_m = ['Top score 1', 'FS', 'MWEdiff', 'Top score', 'Top score 2']
zdf_t7 = fitter.get_trainable(zdf_bt5).drop(dropcols_m, axis=1)
ddf_t7 = fitter.get_trainable(ddf_bt5).drop(dropcols_m, axis=1)
ddf8_feat_score, ddf8_feat_probs, ddf8_feat_results = fitter.get_fit_results(zdf_t7, zdf['Label'], ddf_t7, ddf_gold['Label'])
mup_m = fitter.multi_results(ddf_bt3, ddf_gold, z_results, z_probs, ddf8_feat_results, ddf8_feat_probs, ['Caps', 'Hassub'],['Quotes'])
f1_score(mup_m['Prediction'], mup_m['Label'], average='macro')

Encoding the languages as One Hot variables doesn't change things.

### Results from BERT model

##### Multilingual model for both EN,PT

In [ ]:
ddf_bert0 = pd.read_csv('models/ZeroShot/0/eval-dev/test_results_None.txt', sep='\t')
ddf_bert0_probs = pd.read_csv('models/ZeroShot/0/eval-dev/test_results_None.txt.probs', sep='\t')

In [ ]:
f1_score(ddf_bert0['prediction'].astype('str'), ddf_gold['Label'], average='macro')

In [ ]:
resdf_dev.loc['BERT'] = ['BERT', f1_score(ddf_bert0['prediction'].astype('str'), ddf_gold['Label'], average='macro')]

##### English model for English, multilingual (full) model for PT

In [ ]:
ddf_bert1 = pd.read_csv('models/ZeroShot/1/eval-dev/test_results_None.txt', sep='\t')
ddf_bert1_probs = pd.read_csv('models/ZeroShot/1/eval-dev/test_results_None.txt.probs', sep='\t')

bert0 is the multilingual model, copy the results from the Enligsh model (bert1) over.

In [ ]:
ddf_bert_comb = ddf_bert0.copy()
ddf_bert_comb_probs = ddf_bert0_probs.copy()
for idx,row in ddf_bert1.iterrows():
    _idx,_pred = row
    ddf_bert_comb.loc[_idx,'prediction'] = _pred
for idx,row in ddf_bert1_probs.iterrows():
    _idx,_pred = row
    ddf_bert_comb_probs.loc[_idx,'prediction'] = _pred

In [ ]:
f1_score(ddf_bert_comb['prediction'].astype('str'), ddf_gold['Label'], average='macro')

##### English model for English, multilingual (PT-only) model for PT

In [ ]:
dres_1 = util.load_df('models/ZeroShot/1/eval-dev/test_results_None.txt', delimiter="\t")
dres_2 = util.load_df('models/ZeroShot/2/eval-dev/test_results_None.txt', delimiter="\t")
dres_1['index'] = dres_1['index'].astype(int)
dres_2['index'] = dres_2['index'].astype(int)
dres_2['index'] += len(dres_1)
dres_3 = pd.concat([dres_1, dres_2], ignore_index=True)

In [ ]:
dres_1_p = util.load_df('models/ZeroShot/1/eval-dev/test_results_None.txt.probs', delimiter="\t")
dres_2_p = util.load_df('models/ZeroShot/2/eval-dev/test_results_None.txt.probs', delimiter="\t")
dres_1_p['index'] = dres_1_p['index'].astype(float)
dres_2_p['index'] = dres_2_p['index'].astype(float)
dres_2_p['index'] += len(dres_1_p)
dres_3_p = pd.concat([dres_1_p, dres_2_p], ignore_index=True)

In [ ]:
f1_score(dres_3['prediction'].astype('str'), ddf_gold['Label'], average='macro')

In [ ]:
resdf_dev.loc['BERT_multi1'] = ['BERT multilingual, separate, PT results from full model', f1_score(ddf_bert_comb['prediction'].astype('str'), ddf_gold['Label'], average='macro')]
resdf_dev.loc['BERT_multi2'] = ['BERT multilingual, separate', f1_score(dres_3['prediction'].astype('str'), ddf_gold['Label'], average='macro')]

In [ ]:
resdf_dev

#### Combine BERT with feature model

In [ ]:
mup0 = fitter.multi_results(ddf_bt3, ddf_gold, ddf_bert0['prediction'].astype('str'), ddf_bert0_probs['probs'],
                            ddf5_feat_results, ddf5_feat_probs,
                            ['Caps', 'Hassub'],
                            ['Quotes'], agreeonly=True)
f1_score(mup0['Prediction'], mup0['Label'], average='macro')

##### Combine BERT (en+pt) with feature model

In [ ]:
mup1 = fitter.multi_results(ddf_bt3, ddf_gold, ddf_bert_comb['prediction'].astype('str'),
                            ddf_bert_comb_probs['probs'], ddf5_feat_results, ddf5_feat_probs,
                            ['Caps', 'Hassub'],
                            ['Quotes'])
f1_score(mup1['Prediction'], mup1['Label'], average='macro')

In [ ]:
ddf_res2 = ddf_sub.copy()
ddf_res2.loc[ddf_res2['Setting'] == 'zero_shot', 'Label'] = mup1['Prediction']
ddf_res2.to_csv('data/ddf_sub_20220121_2.csv', index=False)

In [ ]:
mup2 = fitter.multi_results(ddf_bt3, ddf_gold, ddf_bert_comb['prediction'].astype('str'),
                            ddf_bert_comb_probs['probs'], ddf5_feat_results, ddf5_feat_probs,
                            ['Caps', 'Hassub'],
                            ['Quotes'], agreeonly=True)
f1_score(mup2['Prediction'], mup2['Label'], average='macro')

In [ ]:
mup3 = fitter.multi_results(ddf_bt3, ddf_gold, ddf_bert_comb['prediction'].astype('str'),
                            ddf_bert_comb_probs['probs'], ddf5_feat_results, ddf5_feat_probs,
                            ['Caps', 'Hassub'],
                            ['Quotes', '!Trans'])
f1_score(mup3['Prediction'], mup3['Label'], average='macro')

In [ ]:
mup4 = fitter.multi_results(ddf_bt3, ddf_gold, ddf_bert_comb['prediction'].astype('str'),
                            ddf_bert_comb_probs['probs'], ddf5_feat_results, ddf5_feat_probs,
                            ['Caps', 'Hassub'],
                            ['Quotes', '!Trans'], agreeonly=True)
f1_score(mup4['Prediction'], mup4['Label'], average='macro')

In [ ]:
mup5 = fitter.multi_results(ddf_bt3, ddf_gold, dres_3['prediction'].astype('str'),
                            dres_3_p['probs'].astype('float'), ddf5_feat_results, ddf5_feat_probs,
                            ['Caps', 'Hassub'],
                            ['Quotes', '!Trans'], agreeonly=True)
f1_score(mup5['Prediction'], mup5['Label'], average='macro')

In the end, using multilingual model only for Portuguese does not improve things.

In [ ]:
resdf_dev.loc['BERT_feat'] = ['BERT + feature', f1_score(mup0['Prediction'], mup0['Label'], average='macro')]
resdf_dev.loc['BERT_multi_feat'] = ['BERT (multi) + feature', f1_score(mup1['Prediction'], mup1['Label'], average='macro')]
resdf_dev.loc['BERT_multi_feat_agree'] = ['BERT (multi) + feature (agreeonly)', f1_score(mup2['Prediction'], mup2['Label'], average='macro')]
resdf_dev.loc['BERT_multi_feat_trans'] = ['BERT (multi) + feature + trans', f1_score(mup3['Prediction'], mup3['Label'], average='macro')]
resdf_dev.loc['BERT_multi_feat_trans_agree'] = ['BERT (multi) + feature + trans (agreeonly)', f1_score(mup4['Prediction'], mup4['Label'], average='macro')]
resdf_dev.loc['BERT_multi2_feat_trans_agree'] = ['BERT (multi2) + feature + trans (agreeonly)', f1_score(mup5['Prediction'], mup4['Label'], average='macro')]

In [ ]:
resdf_dev

### Eval results

In [ ]:
edf_bert0 = pd.read_csv('models/ZeroShot/0/eval-eval/test_results_None.txt', sep='\t')
edf_bert0_probs = pd.read_csv('models/ZeroShot/0/eval-eval/test_results_None.txt.probs', sep='\t')

In [ ]:
ez_score, ez_probs, ez_results = fitter.get_fit_results(z_emb_multi, zdf['Label'], e_emb_multi)

In [ ]:
dropcols3 = ['Top score 1', 'FS', 'MWEdiff']
zdf_et3 = fitter.get_trainable(zdf_bt3).drop(dropcols3, axis=1)
edf_et3 = fitter.get_trainable(edf_bt3).drop(dropcols3, axis=1)

edf_feat_score3, edf_feat_probs3, edf_feat_results3 = fitter.get_fit_results(zdf_et3, zdf['Label'], edf_et3)
edf_comb3 = fitter.multi_results(edf_bt3, None, ez_results, ez_probs, edf_feat_results3, edf_feat_probs3, ['Caps','Hassub'],['Quotes','!Trans'])
edf_comb3.groupby(['Language','Prediction'])['ID'].count()

In [ ]:
edf_res1 = edf_sub.copy()
edf_res1.loc[edf_res1['Setting'] == 'zero_shot', 'Label'] = edf_comb3['Prediction']
edf_res1.to_csv('data/eval_sub_20220113_sft.csv', index=False)

In [ ]:
edf_comb4 = fitter.multi_results(edf_bt3, None, ez_results, ez_probs, edf_feat_results3, edf_feat_probs3, ['Caps','Hassub'],['Quotes'])
edf_res2 = edf_sub.copy()
edf_res2.loc[edf_res1['Setting'] == 'zero_shot', 'Label'] = edf_comb4['Prediction']
# edf_res2.to_csv('data/eval_sub_20220113_sf.csv', index=False)

In [ ]:
edf_comb_bert1 = fitter.multi_results(edf_bt3, None, edf_bert0['prediction'].astype('str'), edf_bert0_probs['probs'],
                                      edf_feat_results3, edf_feat_probs3,
                                      ['Caps','Hassub'],['Quotes','!Trans'])

In [ ]:
edf_res_bert1 = edf_sub.copy()
edf_res_bert1.loc[edf_res_bert1['Setting'] == 'zero_shot', 'Label'] = edf_comb_bert1['Prediction']
# edf_res_bert1.to_csv('data/eval_sub_20220113_1.csv', index=False)

This one gets a result of 0.6711207561.

In [ ]:
edf_comb_bert1p = fitter.multi_results(edf_bt3, None, edf_bert0['prediction'].astype('str'), edf_bert0_probs['probs'],
                                       edf_feat_results3, edf_feat_probs3,
                                       ['Caps','Hassub'],['Quotes'])

In [ ]:
edf_res_bert1p = edf_sub.copy()
edf_res_bert1p.loc[edf_res_bert1p['Setting'] == 'zero_shot', 'Label'] = edf_comb_bert1p['Prediction']
# edf_res_bert1p.to_csv('data/eval_sub_20220113_6.csv', index=False)

In [ ]:
edf_comb_bert2 = fitter.multi_results(edf_bt3, None, edf_bert0['prediction'].astype('str'),
                                      edf_bert0_probs['probs'], edf_feat_results3, edf_feat_probs3,
                                      ['Caps','Hassub'],['Quotes'], agreeonly=True)

In [ ]:
edf_res_bert2 = edf_sub.copy()
edf_res_bert2.loc[edf_res_bert1['Setting'] == 'zero_shot', 'Label'] = edf_comb_bert2['Prediction']
# edf_res_bert2.to_csv('data/eval_sub_20220113_2.csv', index=False)

This model finally gets a better result than baseline (72.3% vs 70.2%). This essentially means:
 - use the prediction if models agree
 - if they disagree
   - use the BERT base model (because of confidence) except when boolean features Quotes, Caps or Hassub say otherwise.
   - Trans feature (idiomatic interpretation if Trans==False) didn't turn out to be useful.
   
However, using the base BERT model (English for English and multilingual for Portuguese) is a just a tiny bit higher (72.3%).

In [ ]:
edf_bert1 = pd.read_csv('models/ZeroShot/1/eval-eval/test_results_None.txt', sep='\t')
edf_bert1_probs = pd.read_csv('models/ZeroShot/1/eval-eval/test_results_None.txt.probs', sep='\t')

In [ ]:
edf_bert_comb = edf_bert0.copy()
edf_bert_comb_probs = edf_bert0_probs.copy()
for idx,row in edf_bert1.iterrows():
    _idx,_pred = row
    edf_bert_comb.loc[_idx,'prediction'] = _pred
for idx,row in edf_bert1_probs.iterrows():
    _idx,_pred = row
    edf_bert_comb_probs.loc[_idx,'prediction'] = _pred

In [ ]:
edf_comb_bert3 = fitter.multi_results(edf_bt3, None, 
                                      edf_bert_comb['prediction'].astype('str'), edf_bert_comb_probs['probs'], 
                                      edf_feat_results3, edf_feat_probs3,
                                      ['Caps','Hassub'],['Quotes'])

In [ ]:
edf_comb_bert4 = fitter.multi_results(edf_bt3, None, 
                                      edf_bert_comb['prediction'].astype('str'), edf_bert_comb_probs['probs'], 
                                      edf_feat_results3, edf_feat_probs3,
                                      ['Caps','Hassub'],['Quotes'], agreeonly=True)

In [ ]:
edf_res_bert3 = edf_sub.copy()
edf_res_bert3.loc[edf_res_bert3['Setting'] == 'zero_shot', 'Label'] = edf_comb_bert3['Prediction']
# edf_res_bert3.to_csv('data/eval_sub_20220113_3.csv', index=False)

In [ ]:
edf_res_bert4 = edf_sub.copy()
edf_res_bert4.loc[edf_res_bert4['Setting'] == 'zero_shot', 'Label'] = edf_comb_bert4['Prediction']
# edf_res_bert4.to_csv('data/eval_sub_20220113_5.csv', index=False)

Using the English BERT for English improves the result a little bit at 72.5%.

In [ ]:
resdf_eval = pd.DataFrame(columns=['Name', 'Score'])

In [ ]:
resdf_eval.loc['sbert'] = ['Sentence transformers', 0.5580687255]
resdf_eval.loc['sbert_feat'] = ['Sentence transformers + feature', 0.6459566501]
resdf_eval.loc['BERT'] = ['BERT', 0.7024610232]
resdf_eval.loc['BERT_multi'] = ['BERT (multi)', 0.7229592232]
resdf_eval.loc['BERT_feat_trans'] = ['BERT + feature + trans', 0.6711207561]
resdf_eval.loc['BERT_feat'] = ['BERT + feature', 0.7144442603]
resdf_eval.loc['BERT_feat_agree'] = ['BERT + feature (agree)', 0.7229308218]
resdf_eval.loc['BERT_multi_feat'] = ['BERT (multi) + feature', 0.7202048507]
resdf_eval.loc['BERT_multi_feat_agree'] = ['BERT (multi) + feature (agree)', 0.7252418204]

In [ ]:
resdf_eval

### Test results

In [ ]:
tdf_bert0 = pd.read_csv('models/ZeroShot/0/eval-test/test_results_None.txt', sep='\t')
tdf_bert0_probs = pd.read_csv('models/ZeroShot/0/eval-test/test_results_None.txt.probs', sep='\t')

#### Sentence transformers + features

In [ ]:
tz_score, tz_probs, tz_results = fitter.get_fit_results(z_emb_multi, zdf['Label'], t_emb_multi)

In [ ]:
dropcols4 = ['Top score 1', 'FS', 'MWEdiff']
zdf_tt1 = fitter.get_trainable(zdf_bt3).drop(dropcols3, axis=1)
tdf_tt1 = fitter.get_trainable(tdf_bt3).drop(dropcols3, axis=1)

tdf_feat_score, tdf_feat_probs, tdf_feat_results = fitter.get_fit_results(zdf_tt1, zdf['Label'], tdf_tt1)
tdf_comb = fitter.multi_results(tdf_bt3, None, tz_results, tz_probs, tdf_feat_results, tdf_feat_probs, ['Caps','Hassub'],['Quotes'])
tdf_comb.groupby(['Language','Prediction'])['ID'].count()

In [ ]:
tdf_res1 = tdf_sub.copy()
tdf_res1.loc[tdf_res1['Setting'] == 'zero_shot', 'Label'] = tdf_comb['Prediction']
# tdf_res1.to_csv('data/test_sub_20220114_1.csv', index=False)

#### Baseline BERT + features

In [ ]:
tdf_comb_bert = fitter.multi_results(tdf_bt3, None, tdf_bert0['prediction'].astype('str'), tdf_bert0_probs['probs'],
                                     tdf_feat_results, tdf_feat_probs,
                                     ['Caps','Hassub'],['Quotes'])
tdf_comb_bert.groupby(['Language','Prediction'])['ID'].count()

In [ ]:
tdf_res2 = tdf_sub.copy()
tdf_res2.loc[tdf_res2['Setting'] == 'zero_shot', 'Label'] = tdf_comb_bert['Prediction']
# tdf_res2.to_csv('data/test_sub_20220114_2.csv', index=False)

In [ ]:
tdf_comb_bert2 = fitter.multi_results(tdf_bt3, None, tdf_bert0['prediction'].astype('str'), tdf_bert0_probs['probs'],
                                      tdf_feat_results, tdf_feat_probs,
                                      ['Caps','Hassub'],['Quotes'], agreeonly=True)
tdf_comb_bert2.groupby(['Language','Prediction'])['ID'].count()

In [ ]:
tdf_res3 = tdf_sub.copy()
tdf_res3.loc[tdf_res3['Setting'] == 'zero_shot', 'Label'] = tdf_comb_bert2['Prediction']
# tdf_res3.to_csv('data/test_sub_20220114_3.csv', index=False)

#### Baseline BERT (multi) + features

In [ ]:
tdf_bert1 = pd.read_csv('models/ZeroShot/1/eval-test/test_results_None.txt', sep='\t')
tdf_bert1_probs = pd.read_csv('models/ZeroShot/1/eval-test/test_results_None.txt.probs', sep='\t')

In [ ]:
tdf_bert_comb = tdf_bert0.copy()
tdf_bert_comb_probs = tdf_bert0_probs.copy()
for idx,row in tdf_bert1.iterrows():
    _idx,_pred = row
    tdf_bert_comb.loc[_idx,'prediction'] = _pred
for idx,row in tdf_bert1_probs.iterrows():
    _idx,_pred = row
    tdf_bert_comb_probs.loc[_idx,'prediction'] = _pred

In [ ]:
tdf_comb_bert3 = fitter.multi_results(tdf_bt3, None, 
                                      tdf_bert_comb['prediction'].astype('str'), tdf_bert_comb_probs['probs'], 
                                      tdf_feat_results, tdf_feat_probs,
                                      ['Caps','Hassub'],['Quotes'])
tdf_comb_bert3.groupby(['Language','Prediction'])['ID'].count()

In [ ]:
tdf_res4 = tdf_sub.copy()
tdf_res4.loc[tdf_res4['Setting'] == 'zero_shot', 'Label'] = tdf_comb_bert3['Prediction']
# tdf_res4.to_csv('data/test_sub_20220114_4.csv', index=False)

In [ ]:
tdf_comb_bert4 = fitter.multi_results(tdf_bt3, None, 
                                      tdf_bert_comb['prediction'].astype('str'), tdf_bert_comb_probs['probs'], 
                                      tdf_feat_results, tdf_feat_probs,
                                      ['Caps','Hassub'],['Quotes'], agreeonly=True)
tdf_comb_bert4.groupby(['Language','Prediction'])['ID'].count()

In [ ]:
tdf_res5 = tdf_sub.copy()
tdf_res5.loc[tdf_res4['Setting'] == 'zero_shot', 'Label'] = tdf_comb_bert4['Prediction']
# tdf_res5.to_csv('data/test_sub_20220114_5.csv', index=False)

In [ ]:
tdf_comb_bert4[tdf_comb_bert4['Prediction'] != tdf_comb_bert2['Prediction']]

In [ ]:
tdf_comb_bert4[tdf_comb_bert4['Prediction'] != tdf_comb_bert3['Prediction']]